Script Purpose: Scrape cafes' reviews from Google Map

Coding Reference: https://www.linkedin.com/pulse/webscrape-google-map-reviews-using-selenium-python-choy-siew-wearn/?trk=pulse-article_more-articles_related-content-card

List of cafes:
- Close to the beaches (Queen St E.)
    - Bud's Coffee
    - Juice & Java Cafe
    - The Beacher Cafe
    - Simple Coffee
    - Remarkable Beans
    - The Haven Low-Carb Cafe
- Financial District (Bay St.)
    - Dineen Coffee Co.
    - Mos Mos Coffee
    - NEO COFFEE BAR
    - Coffee Island
    - White Rabbit Caffe
    - Dispatch Coffee


In [3]:
import pandas as pd
import time

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

In [2]:
# Create the function for getting reviews through url

def getReviews(cafe_name,url):
    import time 
    
    # Initialize WebDriver
    driver = webdriver.Chrome()

    # Specify the name here
    resturant_name = cafe_name

    # Replace the URL here
    url = url
    print(url)
    driver.get(url)

    #time.sleep(5)
    # Get the total number of reviews
    num_reviews = driver.find_element("xpath","//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[2]/div/div[2]/div[2]").text
    num_reviews = int(num_reviews.split(' ')[0].replace(",",''))

    # Click Sort button and select "Newest"
    # Page Source - Inspect - Locate the button and copy XPATH
    driver.find_element("xpath",'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[7]/div[2]/button').click()
    time.sleep(1)

    driver.find_element("xpath","//*[@id='action-menu']/div[2]").click()
    time.sleep(5)

    # Scoll the review pages from top to bottom
    SCROLL_PAUSE_TIME = 5
    number = 0

    while True:
        number = number+1

        # Scroll down to bottom    
        ele = driver.find_element('xpath','//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
        driver.execute_script('arguments[0].scrollBy(0, 5000);', ele)

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        if number >= num_reviews/10:
            break

    # Store all reviews elements
    item = driver.find_elements("xpath",'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]')
    time.sleep(3)

    # Initialize dataframe for reviews
    review_df = pd.DataFrame(index=range(1000),columns=['name','time','stars','reviews'])

    idx = 0
    for i in item:
        button = i.find_elements(By.TAG_NAME,'button')
        for m in button:
            if m.text == "More":
                m.click()
        time.sleep(5)

        name = i.find_elements(By.CLASS_NAME,"d4r55")
        time = i.find_elements(By.CLASS_NAME,"kvMYJc")
        stars = i.find_elements(By.CLASS_NAME,"wiI7pd")
        reviews = i.find_elements(By.CLASS_NAME,"rsqaWe")

        for j,k,l,p in zip(name,time,stars,reviews):
            review_df['name'][idx] = j.text
            review_df['time'][idx] = p.text
            review_df['stars'][idx] = k.get_attribute("aria-label")
            review_df['reviews'][idx] = l.text
        
            idx = idx + 1
    
    review_df.to_csv('google_review_{}.csv'.format(resturant_name),index=False)


    return review_df

In [3]:
# Read the list of cafes and their reviews URL
urls = pd.read_excel('CafesURL.xlsx')
urls.head()

,Cafe,Street,Review URL
0,Bud's Coffee,Queen St E,https://www.google.ca/maps/place/Bud's+Coffee/...
1,Juice & Java Cafe,Queen St E,https://www.google.ca/maps/place/Juice+%26+Jav...
2,The Beacher Cafe,Queen St E,https://www.google.ca/maps/place/The+Beacher+C...
3,Simple Coffee,Queen St E,https://www.google.ca/maps/place/Simple+Coffee...
4,Remarkable Beans,Queen St E,https://www.google.ca/maps/place/Remarkable+Be...


In [6]:
for i in range(5,len(urls)):
    cafe_name = urls['Cafe'][i]
    cafe_url = urls['Review URL'][i]

    df = getReviews(cafe_name=cafe_name, url=cafe_url)
    df.head()

https://www.google.ca/maps/place/The+Haven+Low-Carb+Cafe/@43.6731409,-79.2887343,17z/data=!4m8!3m7!1s0x89d4cfd9957d5bb3:0xe9a93bbc3145dac!8m2!3d43.6731409!4d-79.2865456!9m1!1b1!16s%2Fg%2F11qpfw2c4s
https://www.google.ca/maps/place/Dineen+Coffee+Co./@43.6481326,-79.3816976,17z/data=!4m8!3m7!1s0x89d4cbb2049c3135:0x5392cad6809f0c78!8m2!3d43.6481326!4d-79.3795089!9m1!1b1!16s%2Fg%2F11g0gwsgvb
https://www.google.ca/maps/place/Mos+Mos+Coffee+%26+Cookie+Shop/@43.6576435,-79.3842556,17z/data=!4m8!3m7!1s0x882b3531a916b775:0x40c1b2e3f50b4c68!8m2!3d43.6576435!4d-79.3842556!9m1!1b1!16s%2Fg%2F11h07mtbzg
https://www.google.ca/maps/place/NEO+COFFEE+BAR/@43.6601367,-79.3880579,17z/data=!4m8!3m7!1s0x882b35ca7fa5ba63:0xd16c33763cf90df4!8m2!3d43.6601367!4d-79.3858692!9m1!1b1!16s%2Fg%2F11h_vx0h1n
https://www.google.ca/maps/place/Coffee+Island/@43.664212,-79.3889548,17z/data=!4m8!3m7!1s0x882b34b14e1b96e5:0xe176339186f4cf1e!8m2!3d43.664212!4d-79.3867661!9m1!1b1!16s%2Fg%2F11dybbxv9c
https://www.google.ca/maps

In [28]:
import os 
import glob

path = os.getcwd()
csv_files = glob.glob(os.path.join(path,"*.csv"))

merged_df = pd.DataFrame()
for f in csv_files:
    df = pd.read_csv(f)
    df['Cafe'] = f.split("\\")[-1].split('_')[-1].split('.')[0]

    print('Location:', f)
    print('File Name:', f.split("\\")[-1])

    merged_df = pd.concat([merged_df,df], ignore_index=True)

Location: c:\Users\shans\OneDrive\Desktop\GitHub\google-resturant-reviews-analysis\google_review_Bud's Coffee.csv
File Name: google_review_Bud's Coffee.csv
Location: c:\Users\shans\OneDrive\Desktop\GitHub\google-resturant-reviews-analysis\google_review_Coffee Island.csv
File Name: google_review_Coffee Island.csv
Location: c:\Users\shans\OneDrive\Desktop\GitHub\google-resturant-reviews-analysis\google_review_Dineen Coffee Co.csv
File Name: google_review_Dineen Coffee Co.csv
Location: c:\Users\shans\OneDrive\Desktop\GitHub\google-resturant-reviews-analysis\google_review_Dispatch Coffee.csv
File Name: google_review_Dispatch Coffee.csv
Location: c:\Users\shans\OneDrive\Desktop\GitHub\google-resturant-reviews-analysis\google_review_Juice & Java Cafe.csv
File Name: google_review_Juice & Java Cafe.csv
Location: c:\Users\shans\OneDrive\Desktop\GitHub\google-resturant-reviews-analysis\google_review_Mos Mos Coffee.csv
File Name: google_review_Mos Mos Coffee.csv
Location: c:\Users\shans\OneDrive\

In [30]:
merged_df.tail()

,name,time,stars,reviews,Cafe
11995,NaN,NaN,NaN,NaN,White Rabbit Caffe
11996,NaN,NaN,NaN,NaN,White Rabbit Caffe
11997,NaN,NaN,NaN,NaN,White Rabbit Caffe
11998,NaN,NaN,NaN,NaN,White Rabbit Caffe
11999,NaN,NaN,NaN,NaN,White Rabbit Caffe


In [26]:
merged_df.shape

(12000, 4)

In [31]:
merged_df.to_csv('cafes_reviews.csv')